In [ ]:
#This Jupyter notebook extracts the most informative features i.e. words from the text. 

In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
import string 
from nltk.stem import WordNetLemmatizer


In [2]:
import pandas as pd
df = pd.read_csv("data.csv")

In [3]:
df.head()

,Tweet,Text Label
0,yeah I got 2 backups for all that. I just hate...,Non-Bullying
1,I hate using my BB but love my iPhone. Haven'...,Non-Bullying
2,Get fucking real dude.,Bullying
3,She is as dirty as they come and that crook ...,Bullying
4,why did you fuck it up. I could do it all day...,Bullying


In [4]:
Tweet = []
Labels = []

for row in df["Tweet"]:
    #tokenize words
    words = word_tokenize(row)
    #remove punctuations
    clean_words = [word.lower() for word in words if word not in set(string.punctuation)]
    #remove stop words
    english_stops = set(stopwords.words('english'))
    characters_to_remove = ["''",'``',"rt","https","’","“","”","\u200b","--","n't","'s","...","//t.c" ]
    clean_words = [word for word in clean_words if word not in english_stops]
    clean_words = [word for word in clean_words if word not in set(characters_to_remove)]
    #Lematise words
    wordnet_lemmatizer = WordNetLemmatizer()
    lemma_list = [wordnet_lemmatizer.lemmatize(word) for word in clean_words]
    Tweet.append(lemma_list)

    for row in df["Text Label"]:
        Labels.append(row)


In [5]:
combined = zip(Tweet, Labels)

In [6]:
def bag_of_words(words):
    return dict([(word, True) for word in words])


In [7]:
#Create new list for modeling
Final_Data = []
for r, v in combined:
    bag_of_words(r)
    Final_Data.append((bag_of_words(r),v))


In [8]:
#Split the data into training and test
from sklearn.model_selection import train_test_split
train_set, test_set= train_test_split(Final_Data, test_size=0.15, random_state=42)

#Naive Bayes for Unigramsm check accuracy
import nltk
import collections
from nltk.metrics.scores import (accuracy, precision, recall, f_measure) 
from nltk import metrics

refsets = collections. defaultdict(set)
testsets = collections.defaultdict(set)

classifier = nltk.NaiveBayesClassifier.train(train_set)
 
for i, (feats, label) in enumerate(test_set):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)

print("Naive Bayes Performance with Unigrams ")    
print("Accuracy:",nltk.classify.accuracy(classifier, test_set))


Naive Bayes Performance with Unigrams 
Accuracy: 0.6662379421221865


In [9]:
#Find most informative features
classifier.show_most_informative_features(n=10)


Most Informative Features
                  fucker = True           Bullyi : Non-Bu =     11.7 : 1.0
                    liar = True           Bullyi : Non-Bu =      7.6 : 1.0
                  vagina = True           Bullyi : Non-Bu =      7.6 : 1.0
                      iq = True           Bullyi : Non-Bu =      7.0 : 1.0
                     low = True           Bullyi : Non-Bu =      6.8 : 1.0
                   worth = True           Non-Bu : Bullyi =      6.6 : 1.0
                  douche = True           Bullyi : Non-Bu =      6.2 : 1.0
                 violent = True           Bullyi : Non-Bu =      6.2 : 1.0
                    tire = True           Bullyi : Non-Bu =      6.2 : 1.0
                     non = True           Bullyi : Non-Bu =      6.2 : 1.0


In [10]:
#Same thing with Bigrams
from nltk import bigrams, trigrams
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures


In [11]:
combined = zip(Tweet,Labels)


In [12]:
#Bag of Words of Bigrams
def bag_of_bigrams_words(words, score_fn=BigramAssocMeasures.chi_sq, n=200):
    bigram_finder = BigramCollocationFinder.from_words(words)  
    bigrams = bigram_finder.nbest(score_fn, n)  
    return bag_of_words(bigrams)


In [13]:
Final_Data2 =[]

for z, e in combined:
    bag_of_bigrams_words(z)
    Final_Data2.append((bag_of_bigrams_words(z),e))


In [14]:
train_set, test_set= train_test_split(Final_Data2, test_size=0.2, random_state=42)
import nltk
import collections
from nltk.metrics.scores import (accuracy, precision, recall, f_measure) 
from nltk import metrics

#Naive Bayes for Bigrams

refsets = collections. defaultdict(set)
testsets = collections.defaultdict(set)

classifier = nltk.NaiveBayesClassifier.train(train_set)

 
for i, (feats, label) in enumerate(test_set):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)


print("Naive Bayes Performance with Bigrams ")    
print("Accuracy:",nltk.classify.accuracy(classifier, test_set))


Naive Bayes Performance with Bigrams 
Accuracy: 0.6164978292329957


In [15]:
classifier.show_most_informative_features(n=10)


Most Informative Features
           ('low', 'iq') = True           Bullyi : Non-Bu =     15.2 : 1.0
       ('piece', 'shit') = True           Bullyi : Non-Bu =     12.0 : 1.0
        ('time', 'http') = True           Bullyi : Non-Bu =      7.6 : 1.0
           ('fuck', 'u') = True           Bullyi : Non-Bu =      7.6 : 1.0
       ('damn', 'homie') = True           Bullyi : Non-Bu =      7.6 : 1.0
        ('suck', 'dick') = True           Bullyi : Non-Bu =      7.6 : 1.0
        ('shut', 'fuck') = True           Bullyi : Non-Bu =      6.8 : 1.0
     ('fucking', 'cunt') = True           Bullyi : Non-Bu =      6.2 : 1.0
        ('holy', 'shit') = True           Bullyi : Non-Bu =      6.2 : 1.0
          ('gay', 'guy') = True           Bullyi : Non-Bu =      5.4 : 1.0


In [16]:
combined = zip(Tweet,Labels)
#Same thing with Trigrams
from nltk import bigrams, trigrams
from nltk.collocations import TrigramCollocationFinder
from nltk.metrics import TrigramAssocMeasures


In [17]:
def bag_of_trigrams_words(words, score_fn=TrigramAssocMeasures.chi_sq, n=200):
    trigram_finder = TrigramCollocationFinder.from_words(words)  
    trigrams = trigram_finder.nbest(score_fn, n)  
    return bag_of_words(trigrams)


In [18]:
Final_Data3 =[]

for z, e in combined:
    bag_of_trigrams_words(z)
    Final_Data3.append((bag_of_trigrams_words(z),e))


In [19]:
train_set, test_set= train_test_split(Final_Data3, test_size=0.15, random_state=42)

#Naive Bayes for Trigrams
import nltk
import collections
from nltk.metrics.scores import (accuracy, precision, recall, f_measure) 
from nltk import metrics


refsets = collections. defaultdict(set)
testsets = collections.defaultdict(set)

classifier = nltk.NaiveBayesClassifier.train(train_set)

 
for i, (feats, label) in enumerate(test_set):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)


print("Naive Bayes Performance with Trigrams ")    
print("Accuracy:",nltk.classify.accuracy(classifier, test_set))


Naive Bayes Performance with Trigrams 
Accuracy: 0.670096463022508


In [20]:
classifier.show_most_informative_features(n=10)


Most Informative Features
('worthless', 'piece', 'shit') = True           Bullyi : Non-Bu =      7.0 : 1.0
   ('oh', 'man', 'suck') = True           Bullyi : Non-Bu =      3.4 : 1.0
('pretty', 'damn', 'awesome') = True           Bullyi : Non-Bu =      3.4 : 1.0
   ('got', 'ta', 'hate') = True           Bullyi : Non-Bu =      3.4 : 1.0
   ('aw', 'man', 'suck') = True           Bullyi : Non-Bu =      3.4 : 1.0
   ('wan', 'na', 'fuck') = True           Bullyi : Non-Bu =      2.9 : 1.0
 ('suck', 'big', 'time') = True           Bullyi : Non-Bu =      2.9 : 1.0
    ("'re", 'gon', 'na') = True           Non-Bu : Bullyi =      2.7 : 1.0
    ('gon', 'na', 'get') = True           Non-Bu : Bullyi =      2.4 : 1.0
('happy', 'new', 'year') = True           Non-Bu : Bullyi =      2.1 : 1.0


In [21]:
combined = zip(Tweet,Labels)


In [22]:
#Combine both unigrams, bigrams, and trigrams

# Import Bigram metrics - we will use these to identify the top 200 bigrams
def bigrams_words(words, score_fn=BigramAssocMeasures.chi_sq,
n=200):
    bigram_finder = BigramCollocationFinder.from_words(words)
    bigrams = bigram_finder.nbest(score_fn, n)
    return bigrams

from nltk.collocations import TrigramCollocationFinder

# Import Bigram metrics - we will use these to identify the top 200 trigrams 
from nltk.metrics import TrigramAssocMeasures

def trigrams_words(words, score_fn=TrigramAssocMeasures.chi_sq,
n=200):
    trigram_finder = TrigramCollocationFinder.from_words(words)
    trigrams = trigram_finder.nbest(score_fn, n)
    return trigrams

#bag of ngrams
def bag_of_Ngrams_words(words):
    bigramBag = bigrams_words(words)
    
    #The following two for loops convert tuple into string
    for b in range(0,len(bigramBag)):
        bigramBag[b]=' '.join(bigramBag[b])
   
    trigramBag = trigrams_words(words)
    for t in range(0,len(trigramBag)):
        trigramBag[t]=' '.join(trigramBag[t])

    return bag_of_words(trigramBag + bigramBag + words)


In [23]:
Final_Data4 =[]

for z, e in combined:
    bag_of_Ngrams_words(z)
    Final_Data4.append((bag_of_Ngrams_words(z),e))


In [24]:
train_set, test_set= train_test_split(Final_Data3, test_size=0.15, random_state=42)

import nltk
import collections
from nltk.metrics.scores import (accuracy, precision, recall, f_measure) 
from nltk import metrics
#Naive Bayes for Ngrams

refsets = collections. defaultdict(set)
testsets = collections.defaultdict(set)

classifier = nltk.NaiveBayesClassifier.train(train_set)

 
for i, (feats, label) in enumerate(test_set):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)


print("Naive Bayes Performance with Ngrams ")    
print("Accuracy:",nltk.classify.accuracy(classifier, test_set))


Naive Bayes Performance with Ngrams 
Accuracy: 0.670096463022508


In [25]:
classifier.show_most_informative_features(n=10)


Most Informative Features
('worthless', 'piece', 'shit') = True           Bullyi : Non-Bu =      7.0 : 1.0
   ('oh', 'man', 'suck') = True           Bullyi : Non-Bu =      3.4 : 1.0
('pretty', 'damn', 'awesome') = True           Bullyi : Non-Bu =      3.4 : 1.0
   ('got', 'ta', 'hate') = True           Bullyi : Non-Bu =      3.4 : 1.0
   ('aw', 'man', 'suck') = True           Bullyi : Non-Bu =      3.4 : 1.0
   ('wan', 'na', 'fuck') = True           Bullyi : Non-Bu =      2.9 : 1.0
 ('suck', 'big', 'time') = True           Bullyi : Non-Bu =      2.9 : 1.0
    ("'re", 'gon', 'na') = True           Non-Bu : Bullyi =      2.7 : 1.0
    ('gon', 'na', 'get') = True           Non-Bu : Bullyi =      2.4 : 1.0
('happy', 'new', 'year') = True           Non-Bu : Bullyi =      2.1 : 1.0
